In [15]:
import pandas as pd
import random
from datetime import datetime, timedelta

# List of US holidays in 2024
us_holidays = [datetime(2024, 1, 1), datetime(2024, 5, 27), datetime(2024, 7, 4), datetime(2024, 9, 2), datetime(2024, 12, 25)]

# Create a list of dates from 1/1/2024 through 12/15/2024
start_date = datetime(2024, 1, 1)
end_date = datetime(2024, 12, 15)
date_list = [start_date + timedelta(days=i) for i in range((end_date - start_date).days + 1)]

# Filter dates to keep only Tuesday, Wednesday, and Thursday
filtered_dates = [date for date in date_list if date.weekday() in [1, 2, 3]]  # 1: Tuesday, 2: Wednesday, 3: Thursday

# Create a DataFrame
df = pd.DataFrame(filtered_dates, columns=['Date'])

# Add a 'Day of the Week' column
df['Day of the Week'] = df['Date'].dt.strftime('%A')

# Duplicate each row to create "Morning" and "Afternoon" sessions
df = pd.concat([df] * 2, ignore_index=True)

# Shuffle the DataFrame to randomly distribute the "Class" labels across all available dates
random.seed(42)  # Set a seed for reproducibility
df = df.sample(frac=1).reset_index(drop=True)

# Assign "Morning" and "Afternoon" labels in order for each day
df['Session'] = (df.groupby('Date').cumcount() % 2).map({0: 'Morning', 1: 'Afternoon'})

# Assign "One Session" labels randomly
df['Class'] = ['One Session' if random.random() < 0.5 else '' for _ in range(len(df))]

# Sort the DataFrame by date in ascending order and "Day of the Week" in descending order
df = df.sort_values(by=['Date', 'Day of the Week'], ascending=[True, False])

# Add a "4 session" class once per week (120 times)
class_week = 0
class_count = 0
for idx, row in df.iterrows():
    if row['Class'] == '' and class_week == 0:
        df.at[idx, 'Class'] = '4 session'
        class_week = 4  # Allocate the class for four consecutive sessions
        class_count += 1
        if class_count >= 120:  # Stop after allocating 120 classes
            break
    if class_week > 0:
        class_week -= 1

# Reset the index of the DataFrame
df.reset_index(drop=True, inplace=True)

# Calculate the remaining count for "2 Session" classes
remaining_count = 186 - len(df[(df['Class'] == '2 Session') | (df['Class'] == '4 session')])

# Assign "2 Session" labels for remaining rows with sessions on two consecutive days
consecutive_days = 0
for idx, row in df.iterrows():
    if row['Class'] == '' and remaining_count > 0:
        if consecutive_days == 0:
            df.at[idx, 'Class'] = '2 Session'
            consecutive_days = 2  # Allocate the class for two consecutive days
            remaining_count -= 1
    if consecutive_days > 0:
        consecutive_days -= 1

# Assign "Open" label for any remaining available "Class" morning or afternoon sessions
for idx, row in df.iterrows():
    if row['Class'] == '':
        df.at[idx, 'Class'] = 'Open'

# Display the resulting DataFrame
print(df)

# Save the DataFrame to an Excel file
df.to_excel('class schedule.xlsx', index=False)

          Date Day of the Week    Session        Class
0   2024-01-02         Tuesday    Morning  One Session
1   2024-01-02         Tuesday  Afternoon    4 session
2   2024-01-03       Wednesday    Morning  One Session
3   2024-01-03       Wednesday  Afternoon  One Session
4   2024-01-04        Thursday    Morning    2 Session
..         ...             ...        ...          ...
295 2024-12-10         Tuesday  Afternoon  One Session
296 2024-12-11       Wednesday    Morning    2 Session
297 2024-12-11       Wednesday  Afternoon    4 session
298 2024-12-12        Thursday    Morning    2 Session
299 2024-12-12        Thursday  Afternoon  One Session

[300 rows x 4 columns]
